In [3]:
import numpy as np
import pandas as pd
import scanpy as sc
import anndata as ad
import torch
import torch.nn as nn
import torch.optim as optim
import ot
import sys
import os
import tarfile
import scipy.io
import h5py
import gzip
from scipy.sparse import issparse
from scipy.sparse import csr_matrix

[KeOps] Compiling cuda jit compiler engine ... OK
[pyKeOps] Compiling nvrtc binder for python ... OK


In [28]:
adata = sc.read_10x_h5(filename="/workspace/ImputationOT/data/matrix.h5")
print(adata)

/opt/conda/lib/python3.10/site-packages/anndata/_core/anndata.py:1820: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


AnnData object with n_obs × n_vars = 8371 × 18082
    var: 'gene_ids', 'feature_types', 'genome'


/opt/conda/lib/python3.10/site-packages/anndata/_core/anndata.py:1820: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


In [37]:
adata = sc.read_h5ad("/workspace/ImputationOT/data/healthy.h5ad")
print(adata)

AnnData object with n_obs × n_vars = 486134 × 33538
    obs: 'celltype', 'sampleID', 'SampleType', 'Source', 'DonorID', 'Organ', 'Tissue', 'TissueDetail', 'Gender', 'Age', 'AgeBin', 'DeathType', 'protocol', 'Race'
    var: 'gene_ids'
    uns: 'celltype_colors'
    obsm: 'X_umap'


In [29]:
X = adata.X
print("Matrix Shape:", X.shape)
print("Density:", np.count_nonzero(X) / (X.shape[0] * X.shape[1]))
# print("Density:", X.nnz / (X.shape[0] * X.shape[1]))
print("Minimum Value:", X.min())
print("Maximum Value:", X.max())

Matrix Shape: (9409, 60970)
Density: 0.04190173273600859
Minimum Value: 0.0
Maximum Value: 13.540438


In [31]:
print(adata.obs['Batch'])

index
Cell9517     Batch1
Cell5323     Batch1
Cell6864     Batch1
Cell9648     Batch1
Cell1185     Batch1
              ...  
Cell55408    Batch6
Cell52882    Batch6
Cell51973    Batch6
Cell57940    Batch6
Cell53110    Batch6
Name: Batch, Length: 12097, dtype: category
Categories (6, object): ['Batch1', 'Batch2', 'Batch3', 'Batch4', 'Batch5', 'Batch6']
